'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
'''

In [1]:
'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes: 
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    TableName:Lambda Function
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function can take on any form but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
               CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['NEL_Column'],
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}

import SQLA_main as SQLA_main #import main SQLAlchemy functions



Clearing old DB


In [2]:
'''
Define other custom modules

'''
import expression as Expr
import importSpecial as importSpecial #special import functions are defined here
import importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [3]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
#will insert fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)


session.commit()

Reading csv for import to Feasibility Questions

Reading csv record: Feas-1
Adding to variable dictionary: OFFSITE_SD_Exist

Reading csv record: Feas-2
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-3
Adding to variable dictionary: GW_Risk

Reading csv record: Feas-4
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-5
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-6
Adding to variable dictionary: Soil_Type

Reading csv record: Feas-7
Adding to variable dictionary: Count_CB

Reading csv record: Feas-8
Adding to variable dictionary: Runoff_Type

Reading csv record: Feas-9
Adding to variable dictionary: TFMR_Exist

Reading csv record: Feas-10
Adding to variable dictionary: DS_SS_Exist

Reading csv record: Feas-11
Adding to variable dictionary: Fac_Slope

Reading csv record: Feas-12
Adding to variable dictionary: Can_Add_SD

Reading csv record: Feas-13
Adding to variable dictionary: Pave_Area
Adding to variable dictionary: BMP_Size(bas

imported records in  238  rows
associating records...


In [ ]:
#EXPRESSION EVALUATOR:
import pickle
#### DEFINE GLOBALLY SCOPED CONSTANTS TO HELP UNDERSTAND WHAT ARRAY ELEMENT WE'RE ACCESSING:
####  Constants for expression record query tuple:
C_exprRecTuple_exprName = 0
C_exprRecTuple_exprStr = 1
C_exprRecTuple_exprDataType = 2
C_exprRecTuple_Vars = 3

C_VarDict_VarName = 0 #### Constant for Var Dict Key
#### Constants for Var Dict Array:
# use the project standard VarDict: 
#    key = var name: [VarName, VarType, StoredTable, StoredField, QryOnUniqueField, DataType]
C_VarDict_VarName = 0
C_VarDict_VarType = 1
C_VarDict_StoredTable = 2
C_VarDict_StoredField = 3
C_VarDict_QryOnUniqueField = 4
C_VarDict_DataType = 5


def EvalExpr(SQLARec, QryOnUnqFieldValsDict): #### pass in expression record as a sqlalchemy query record
    #QryOnUnqFieldValsDict: dictionary of the value that value obtaining query should query against. FORMAT:
        #{key = table.fieldname: item = unique value}
        #this allows us to define, for example, the unique facility and base-bmp pair to query for:
            #{facility_chars.facility_id: 2,
            # base_bmps.bmp_name: 'hydrodynamic_separator'
            #}
    
#     procstr = exprRecTuple[C_exprRecTuple_exprStr] #get expression string from record tuple
    if exprRecTuple[C_exprRecTuple_Vars] is None: #will be NoneType if no vars were entered to expression record (expression is probably a constant)
#         print ('empty')
        Vars= {} #make empty vars dictionary
    else:
        Vars = pickle.loads(exprRecTuple[C_exprRecTuple_Vars]) #unpickle to Vars variable (Dictionary type)
    print('proccessing expression: ' +  exprRecTuple[C_exprRecTuple_exprName] + '=' + procstr)
    for aVar in Vars.items(): #iterate thru each Var in Vars, replacing procstr's Var instances w/ Var's value 
        procstr = procstr.replace(aVar[C_VarDict_VarName],getVal(dbFileName, aVar,QryOnUnqFieldValsDict))
#         aVal = getVal(dbFileName, aVar,QryOnUnqFieldValsDict)
#         print ('Replace: ' + aVar[C_VarDict_VarName] + 'w ' + aVal + ' in ' + procstr)
#         procstr = procstr.replace (aVar[C_VarDict_VarName],aVal)
#         print ('Result ' + procstr)
        print('proccessing expression: ' +  exprRecTuple[C_exprRecTuple_exprName] + '=' + procstr)
    myVal = eval(procstr)
    print ('  eval(' + str(procstr) + ')=' + str(myVal))
    return myVal

        
def getVal(dbFileName, aVar, QryOnUnqFieldValsDict): #retrieve DB value, or call expression evaluation of passed variable (expects tuple of expression Query)   
    #QryOnUnqFieldValsDict: dictionary of the value that value obtaining query should query against. FORMAT:
        #{key = table.fieldname: item = unique value}
        #this allows us to define, for example, the unique facility and base-bmp pair to query for:
            #{facility_chars.facility_id: 2,
            # base_bmps.bmp_name: 'hydrodynamic_separator'
            #}

    print('    attempting to retrieve value for: ', aVar[C_VarDict_VarName] )
    strdbVal = 'fault_if_still_this'
    
    if aVar[1][C_VarDict_VarType] == 'val': #### value is housed somewhere in database. get value
        #unpack serialized data:
        dbTableName = aVar[1][C_VarDict_StoredTable]
        dbFieldName = aVar[1][C_VarDict_StoredField]
        dbQryOnUniqueField = aVar[1][C_VarDict_QryOnUniqueField]
        UnqFieldValsDict_Key = dbTableName + '.' + dbQryOnUniqueField
#         print ('unq key ' + UnqFieldValsDict_Key)
        #find matching table-field in QryOnUnqFieldValsDict:
        try:
            QryOnUniqueFieldVal = QryOnUnqFieldValsDict[UnqFieldValsDict_Key]#[dbTableName + '.' + dbFieldName]
#             print (str(QryOnUnqFieldValsDict))
        except KeyError:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' for the var was not included with QryOnUnqFieldValsDict')
            return strdbVal
        sql_str = '''SELECT ''' + dbFieldName + ''' FROM ''' + dbTableName + ''' WHERE ''' + dbQryOnUniqueField + '''=''' + str(QryOnUniqueFieldVal)
        print('      QUERY:' + sql_str)       
        myRows = QrySQL(dbFileName, sql_str, ())
        dbVal = myRows[0][0] #### return record as value
        if type(dbVal) == str:
            strdbVal = '\'' + dbVal + '\'' #encapsulate in quotes so python eval reads as str and not var
        else: #assume numeric
            strdbVal = str(dbVal) #### cast to string
        print('       QUERY RESULT: ' + aVar[C_VarDict_VarName] + '=' + strdbVal)
    
    elif aVar[1][C_VarDict_VarType]=='exp':
        print ('      This is an expression. Prepare to re-enter EvalExpr...')
        dbTableName = 'expressions'
        dbFieldName = 'expression_name'
        sql_str = '''SELECT expression_name, expression_str, expression_data_type, vars \
        FROM ''' + dbTableName + ''' WHERE ''' + dbFieldName + '''= \'''' +  aVar[C_VarDict_VarName] + '''\''''
        print('      QUERY:' + sql_str)
        myRows = QrySQL(dbFileName, sql_str, ())
        dbVal = myRows[0][0] #### return record as value
        print('       Reentering EvalExpr....')
        strdbVal = str(EvalExpr(dbVal)) #### cast to string
    
    elif aVar[1][C_VarDict_VarType]=='dxp':
    #                dynamic_expr_format: dyn expr name(exprID_tablename~exprID_fieldname~unqFieldName)
    #                identifies what table and field name holds reference to the expression_id, and the unique field  of the table that identifies the record. 
        print ('     This is a dynamic expression. Query for static expression using provided unique identifiers')
        #unpack dxp info:
        dbTableName = aVar[1][C_VarDict_StoredTable]
        dbFieldName = aVar[1][C_VarDict_StoredField]
        dbQryOnUniqueField = aVar[1][C_VarDict_QryOnUniqueField]
        UnqFieldValsDict_Key = dbTableName + '.' + dbQryOnUniqueField
#         print ('unq key ' + UnqFieldValsDict_Key)
        #get ready to query for expression id:
        #get unique value to query on:
        try:
            QryOnUniqueFieldVal = QryOnUnqFieldValsDict[UnqFieldValsDict_Key]#[dbTableName + '.' + dbFieldName]
            print (str(QryOnUnqFieldValsDict))
        except KeyError:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' for the var was not included with QryOnUnqFieldValsDict')
            return strdbVal
        sql_str = 'SELECT ' + dbFieldName + ' FROM ' + dbTableName + ' WHERE ' + dbQryOnUniqueField + '=(?)' 
        print (sql_str + QryOnUniqueFieldVal)
        ParamTuple = (QryOnUniqueFieldVal,)
        myRows = QrySQL(dbFileName, sql_str, ParamTuple) #query for dynamic expression record
        if len(myRows) == 0:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' dxp exprssion query has no record')
            return strdbVal
        expression_id = myRows[0][0] #extract expression_id
        if expression_id == -1234:
            print ('     FAULT!!!! While evaluating expression: DB stored table name + field name: ' + dbTableName + '.' + dbFieldName + ' dxp exprssion query has no record')
            return strdbVal        
        #now query for static expression record:
        sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
                    FROM expressions WHERE expression_id = (?)'
        ParamTuple = (expression_id,)
        myExpr = QrySQL (dbFileName, sql_str, ParamTuple)
        if len(myExpr)>0:
            myExpr = myExpr[0] #extract tuple from list
            print ('       dynamic expression: ' + aVar[0] + ' = ' + ' static expression: ' + myExpr[0])
            print ('       Reentering EvalExpr...')
            strdbVal = str(EvalExpr(dbFileName,myExpr, QryOnUnqFieldValsDict))        
    else:
        print (strdbVal)
    return strdbVal


In [4]:
#EVALUATE base_bmp feasibility:
# def is_base_bmp_feasible(dbFileName, facility_id, base_bmp_id):
#     #use db feasibility data to determine if base bmp is feasible for the given facility.
#     print ('Determine feasibility of base bmp: ' + str(base_bmp_id) + '  at Facility: ' + str(facility_id))
#     sql_str = 'SELECT bbftd.feasibility_test_question_id, ftq.feas_id, is_feasible, base_bmp_feasibility_test_result_id  \
#                 FROM base_bmp_feasibility_test_results AS bbftr \
#                 INNER JOIN base_bmp_feasibility_test_definitions AS bbftd ON bbftr.base_bmp_feasibility_test_definition_id = bbftd.base_bmp_feasibility_test_definition_id \
#                 INNER JOIN feasibility_test_questions AS ftq ON bbftd.feasibility_test_question_id = ftq.feasibility_test_question_id \
#                 WHERE facility_id = (?) AND base_bmp_id =(?)'
#     ParamTuple = (facility_id,base_bmp_id)
#     Tests = QrySQL (dbFileName, sql_str, ParamTuple)
#     is_bb_feasible = 1 #start w/ bmp feasibility as true (necessary start condition for bool and ops to work correctly)
# #     print ('using DB data:')
#     for Test in Tests:
#         print ('Feas_id: ' + str(Test[1]) + ' Test Result: ' +  str(Test[2]) + ' (bbftd.feasibility_test_question_id ' + str(Test[0]) + ' & bbftr.base_bmp_feasibility_test_result_id '+ str(Test[3]) + ')')
#         is_bb_feasible = is_bb_feasible * Test[2] #bool op to determine bmp feasibility
#     print ('  Result: ' + str(bool(is_bb_feasible)))
#     return is_bb_feasible
    
def Eval_base_bmp_feasibility_tests(myFacility, myBaseBMP):
    #evaluate feasibility tests for a single facility in facility_char table & a single base_bmp from the base_bmps table
    print ('Determine Facility-base_bmp feasibility. Unique Identifiers:   Facility ID: ', 
           myFacility.id, '  base_bmp_ID: ', myBaseBMP.id )
#     sql_str = 'SELECT bbftd.feasibility_test_question_id, ftq.question_expression_id, bbftd.base_bmp_feasibility_test_definition_id \
#                 FROM base_bmp_feasibility_test_definitions AS bbftd \
#                 INNER JOIN feasibility_test_questions AS ftq \
#                 ON bbftd.feasibility_test_question_id = ftq.feasibility_test_question_id \
#                 WHERE base_bmp_id = (?)'
#     ParamTuple = (base_bmp_id,)
#     myRows = QrySQL(dbFileName, sql_str, ParamTuple)
    for row in session.query(BBFTD.feasibility_test_question_id, FTQ.question_expression_id, BBFTD.id).filter(
        BBFTD.feasibility_test_question_id == FTQ.id).filter(BBFTD.base_bmp_id == myBaseBMP.id):
#         #build QryOnUnqFieldValsDict:
        print ('test', row)
        print ('\n Attempting eval of feasibility_test ID: ', row.feasibility_test_question_id)
        QryOnUnqFieldValsDict = {'facility_chars.facility_id': myFacility.id,
                                 'base_bmps.bmp_name':myBaseBMP.bmp_name} #bmp_name is needed b/c the test's expression may be unique to a particular bmp 
        print (QryOnUnqFieldValsDict)
#         sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
#                     FROM expressions WHERE expression_id = (?)'
#         ParamTuple = (row[1],)
#         myExpr = QrySQL (dbFileName, sql_str, ParamTuple)
        #get expression record for the question_expression:
        myExpr = session.query(Expressions).filter(Expressions.id == row.question_expression_id)
        if myExpr.first() is not None: #then record retrieved
            is_feasible = bool(EvalExpr(myExpr, QryOnUnqFieldValsDict))
#         if len(myExpr)>0:
#             myExpr = myExpr[0] #extract tuple from list
#             is_feasible = bool(EvalExpr(dbFileName,myExpr, QryOnUnqFieldValsDict))
#             print ('Writing to DB Feasibility Test Result: ' + str(is_feasible) + '(' + str(int(is_feasible)) + ')')
#             #insert/update feasibility test result:
#             RecordFieldLS = ['facility_id','base_bmp_feasibility_test_definition_id', 'is_feasible']
#             RecordValsTPL = (facility_id, row[2], int(is_feasible))
#             recID = setRecord(dbFileName, 'base_bmp_feasibility_test_results', RecordFieldLS, RecordValsTPL, 'base_bmp_feasibility_test_result_id', \
#                               'facility_id = (?) AND base_bmp_feasibility_test_definition_id = (?)',
#                              (facility_id, row[2]))
#             print ('  Wrote to base_bmp_feasibility_test_results as recordID: ' + str(recID))
#         else:
#             print ('!!!! FAULT! expression_id: ' + row[1] + ' not found in expressions table. this should not happen.')
#             return False
    
    
# def Eval_base_bmp_feasibility_tests(dbFileName, facility_id, base_bmp_id, bmp_name,flgCommitAfterTransaction):
#     #evaluate feasibility tests for a single facility in facility_char table & a single base_bmp from the base_bmps table
#     print ('Determine Facility-base_bmp feasibility. Unique Identifiers:   Facility ID: ' + str(facility_id) + '  base_bmp_ID: ' + str(base_bmp_id) )
#     sql_str = 'SELECT bbftd.feasibility_test_question_id, ftq.question_expression_id, bbftd.base_bmp_feasibility_test_definition_id \
#                 FROM base_bmp_feasibility_test_definitions AS bbftd \
#                 INNER JOIN feasibility_test_questions AS ftq \
#                 ON bbftd.feasibility_test_question_id = ftq.feasibility_test_question_id \
#                 WHERE base_bmp_id = (?)'
#     ParamTuple = (base_bmp_id,)
#     myRows = QrySQL(dbFileName, sql_str, ParamTuple)
    
#     for row in myRows:
#         #build QryOnUnqFieldValsDict:
#         print ('\nattempting eval of feasibility_test: ' + str(row[0]) + '(base_bmp_feas._test ID: ' + str(row[2]) + ')')
#         QryOnUnqFieldValsDict = {'facility_chars.facility_id': facility_id,
#                                  'base_bmps.bmp_name': bmp_name} #bmp_name is needed b/c the test's expression may be unique to a particular bmp 
# #         print (QryOnUnqFieldValsDict)
#         sql_str = 'SELECT expression_name, expression_str, expression_data_type, vars \
#                     FROM expressions WHERE expression_id = (?)'
#         ParamTuple = (row[1],)
#         myExpr = QrySQL (dbFileName, sql_str, ParamTuple)
#         if len(myExpr)>0:
#             myExpr = myExpr[0] #extract tuple from list
#             is_feasible = bool(EvalExpr(dbFileName,myExpr, QryOnUnqFieldValsDict))
#             print ('Writing to DB Feasibility Test Result: ' + str(is_feasible) + '(' + str(int(is_feasible)) + ')')
#             #insert/update feasibility test result:
#             RecordFieldLS = ['facility_id','base_bmp_feasibility_test_definition_id', 'is_feasible']
#             RecordValsTPL = (facility_id, row[2], int(is_feasible))
#             recID = setRecord(dbFileName, 'base_bmp_feasibility_test_results', RecordFieldLS, RecordValsTPL, 'base_bmp_feasibility_test_result_id', \
#                               'facility_id = (?) AND base_bmp_feasibility_test_definition_id = (?)',
#                              (facility_id, row[2]))
#             print ('  Wrote to base_bmp_feasibility_test_results as recordID: ' + str(recID))
#         else:
#             print ('!!!! FAULT! expression_id: ' + row[1] + ' not found in expressions table. this should not happen.')
#             return False

# def Eval_ALL_base_bmp_feasibility_tests(dbFileName, facility_id,flgCommitAfterTransaction):
# #     wrapper around Eval_base_bmp_feasibility_tests. evals facilities in facility_char table
#     print ('\nEvaluating base bmp feasibiilty tests for facility: ' + str(facility_id))
#     sql_str = 'SELECT base_bmp_id, bmp_name FROM base_bmps ORDER BY base_bmp_id'
#     ParamTuple = ()
#     myBMPs = QrySQL(dbFileName, sql_str, ParamTuple)
#     for aBMP in myBMPs:
#         print ('\nCalling eval_base_bmp_feasibility_tests FOR base_bmp: ' + str(aBMP[0]) + ' ' + aBMP[1])
#         Eval_base_bmp_feasibility_tests(dbFileName, facility_id, aBMP[0], aBMP[1],flgCommitAfterTransaction)

        
# dbConnection = getDBConnection('_jonhonda_dat\\special_prj\\StrBMPModelDB003')
# flgCommitAfterTransaction=False
# for row in QrySQL(dbConnection, 'SELECT facility_id FROM facility_chars', ()):
#     Eval_ALL_base_bmp_feasibility_tests(dbConnection,row[0],flgCommitAfterTransaction)
# if flgCommitAfterTransaction==False: dbConnection.commit() #commit transactions at end if false
# dbConnection.close()
myFacility = session.query(Facility_Chars).filter(Facility_Chars.id == 1).first()            
myBMP = session.query(Base_BMPs).filter(Base_BMPs.id == 1).first()
Eval_base_bmp_feasibility_tests(myFacility, myBMP)

Determine Facility-base_bmp feasibility. Unique Identifiers:   Facility ID:  1   base_bmp_ID:  1
test (1, 1, 1)

 Attempting eval of feasibility_test ID:  1
{'facility_chars.facility_id': 1, 'base_bmps.bmp_name': 'Hydrodynamic Separation'}
<Expressions(id='1', expression_name='Feas-1', expression_str='OFFSITE_SD_Exist=='Yes'', expression_data_type='FLOAT', vars='b'\x80\x04\x95L\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x10OFFSITE_SD_Exist\x94]\x94(h\x01\x8c\x03val\x94\x8c\x0efacility_chars\x94h\x01\x8c\x0bfacility_id\x94\x8c\x05FLOAT\x94es.'')>


NameError: name 'EvalExpr' is not defined

In [ ]:
session.close()
engine.dispose()